In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [2]:
def bilinearInterpolation(img, row, col):
    up_idx = int(row)
    down_idx = up_idx + 1
    left_idx = int(col)
    right_idx = left_idx + 1

    up_wight = abs(row - down_idx)
    down_weight = 1 - up_wight
    right_wight = abs(col - left_idx)
    left_wight = 1 - right_wight
    if (up_idx >= 0 and down_idx < img.shape[0]) and (left_idx >= 0 and right_idx < img.shape[1]):
        img_pix = {
            'upleft' : img[up_idx, left_idx],
            'upright' : img[up_idx, right_idx],
            'downleft' : img[down_idx, left_idx],
            'downright' : img[down_idx, right_idx]
        }
        
        up_point = (left_wight * img_pix['upleft']) + (right_wight * img_pix['upright'])
        down_point =(left_wight * img_pix['downleft']) + (right_wight * img_pix['downright'])
        point = (down_weight * down_point) + (up_wight * up_point)
        
        return np.uint16(point)
        
    else:
        return 0

In [3]:
def boundryAffine(img, s):
    row, col = img.shape[0], img.shape[1]
    cords = np.array([[0, 0, 1], [0, col-1, 1], [row-1, 0, 1], [row-1, col-1, 1]])
    cords_prim = np.dot(s, cords.T)
    minR, minC, _ = np.min(cords_prim, axis=1).astype(np.int64)
    maxR, maxC, _ = (np.ceil(np.max(cords_prim, axis=1))).astype(np.int64)
    H = maxR - minR + 1
    W = maxC - minC + 1
    
    return minR, minC, maxR, maxC, H, W

def transformAffine(img, s):
    minR, minC, maxR, maxC, H, W = boundryAffine(img, s)
    img2 = np.zeros((H, W), dtype=np.uint16)
    s_inv = np.linalg.inv(s)
    for i in range (minR, maxR):
        for j in range(minC, maxC):
            p_prim = np.array([i, j, 1])
            new_i, new_j, _ = np.dot(s_inv, p_prim)
            img2[i - minR, j - minC] = bilinearInterpolation(img, new_i, new_j)
    
    return img2

In [4]:
def boundryProjective(img, s):
    row, col = img.shape[0], img.shape[1]
    cords = np.array([[0, 0, 1], [0, col-1, 1], [row-1, 0, 1], [row-1, col-1, 1]])
    cords_prim = np.dot(s, cords.T)
    cords_prim /= cords_prim[2, :]
    minR, minC, _ = np.min(cords_prim, axis=1).astype(np.int64)
    maxR, maxC, _ = (np.ceil(np.max(cords_prim, axis=1))).astype(np.int64)
    H = maxR - minR + 1
    W = maxC - minC + 1
    
    return minR, minC, maxR, maxC, H, W

def transformProjective(img, s):
    minR, minC, maxR, maxC, H, W = boundryProjective(img, s)
    img2 = np.zeros((H, W), dtype=np.uint16)
    s_inv = np.linalg.inv(s)
    for i in range (minR, maxR):
        for j in range(minC, maxC):
            p_prim = np.array([i, j, 1])
            point = np.dot(s_inv, p_prim)
            new_i, new_j, _ = point / point[2]
            img2[i - minR, j - minC] = bilinearInterpolation(img, new_i, new_j)
    
    return img2

In [5]:
def get_point(image, numPts):
    %matplotlib
    fig, ax = plt.subplots(1, figsize=(15, 30))
    plt.imshow(image, cmap='gray')
    pts = np.round(np.array(plt.ginput(n=numPts)))
    # list of tuple -> array first col: X, second col: Y
    pts = pts[:, [1,0]].T  # first row: X, second row: Y
    plt.close()
    
    return pts

In [6]:
img_gray = cv.imread('einstein.jpg', cv.IMREAD_GRAYSCALE)
img_gray_affine = cv.imread('einstein_Affine.jpg', cv.IMREAD_GRAYSCALE)
img_gray_projective = cv.imread('einstein_Projective.jpg', cv.IMREAD_GRAYSCALE)

In [7]:
# points = get_point(img_gray, 3)

### when we have 3 point 
np.vstack([1, 1, 1]) at last row then multyply p_prim(form destination) with invers P
### when we have more than 3 point 
first multypy p and p_prime with p.T then we have 3x3 matrices then resume the same as befor

In [14]:
p = np.array([[402, 400, 479], [320, 499, 443]])
p_prime = np.array([[487, 471, 569],[389, 530, 515]])
one = np.ones((1, 3))

p = np.vstack((p, one))
p_prime = np.vstack((p_prime, one))

In [17]:
np.linalg.det(p)

-14029.000000000016

In [19]:
s = p_prime.dot(np.linalg.inv(p))
s

array([[ 1.18654216e+00, -7.61280205e-02,  3.43710172e+01],
       [ 3.71444864e-01,  7.91859719e-01, -1.37159455e+01],
       [ 0.00000000e+00,  3.52365706e-19,  1.00000000e+00]])

In [27]:
p = np.array([[402, 400, 479, 123, 564], [320, 499, 443, 234, 543]])
p_prime = np.array([[487, 471, 569, 234, 512],[389, 530, 515, 234, 342]])
one = np.ones((1, p.shape[1]))

p = np.vstack((p, one))
p_prime = np.vstack((p_prime, one))

In [29]:
p_new = np.dot(p, p.T)
p_prime_new = np.dot(p_prime, p.T)

In [30]:
s = p_prime_new.dot(np.linalg.inv(p_new))
s

array([[ 8.86484990e-01, -2.41280978e-01,  2.04073891e+02],
       [ 1.24595771e-01,  4.37917130e-01,  1.74376499e+02],
       [-3.46944695e-18, -6.93889390e-18,  1.00000000e+00]])